## 用自然语言和文档聊天

In [3]:
# 导入必须的包
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredExcelLoader
from langchain.text_splitter import CharacterTextSplitter

# from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

# 引入Chat model
from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

# 定义ChatDoc
class ChatDoc():
  def __init__(self):
    self.doc = None
    self.splitText = []
    self.template = [
      ("system", "你是一个合同草拟的专家，你从不说自己是大模型或智能助手, \
                  你会根据下面提供的上下文内容来回答我的问题。\n 上下文内容 \n {context} \n"),
      ("human", "你好！"),
      ("ai", "你好！"),
      ("human", "{question}")
    ]
    self.prompt = ChatPromptTemplate.from_messages(self.template)

  def getFile(self):
    doc = self.doc
    loaders = {
      "docx":Docx2txtLoader,
      "pdf":PyPDFLoader,
      "xlsx":UnstructuredExcelLoader
    }
    file_extension = doc.split(".")[-1]
    loader_class = loaders.get(file_extension)
    if loader_class:
      try:
        loader = loader_class(doc)
        text = loader.load()
        return text
      except Exception as e:
        print(f"Error Loading {file_extension} files: {e}")
    else:
      print(f"Unsupported Loading {file_extension} files: {e}")
      return None

  # 文档分割
  def splitSentences(self):
    full_text = self.getFile()
    if full_text != None :
      # 对文档进行切割
      text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
      texts = text_splitter.split_documents(full_text)
      self.splitText = texts

  # 向量化与向量存储
  def embeddingAndVectorDB(self):
    # embeddings = OpenAIEmbeddings()

    embeddings = OllamaEmbeddings(
      base_url="http://localhost:11434",
      model="nomic-embed-text:latest"
    )
    chroma_db = Chroma.from_documents(
      documents=self.splitText,
      embedding=embeddings
    )
    return chroma_db

  # 提问并找到相关的文本块
  def askAndFindFiles(self, question):
    db = self.embeddingAndVectorDB()
    # retriever = db.as_retriever(search_type='mmr')
    retriever = db.as_retriever(search_type="similarity_score_threshold",
                                search_kwargs = {"score_threshold":.5, "k":1})
    return retriever.get_relevant_documents(query=question)

  # Chat with Document By 自然语言
  def chatWithDoc(self, question):
    _content = ""
    context = self.askAndFindFiles(question)
    for i in context:
      _content += i.page_content

    messages = self.prompt.format_messages(context=_content, question=question)
    chat = ChatOllama(
      model="llama3.1:8b"
    )
    return chat.invoke(messages)


chat_doc = ChatDoc()
chat_doc.doc = "files/file.docx"
chat_doc.splitSentences()
remsg = chat_doc.chatWithDoc("合同纠纷怎么处理呢？")
print("-----------------")
print(remsg.content)

/opt/homebrew/Caskroom/miniconda/base/envs/rag/lib/python3.13/site-packages/langchain_core/vectorstores/base.py:1077: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': 'files/file.docx'}, page_content='第七条 争议解决\n\n合同履行中发生争议，双方应协商解决；协商不成可向乙方所在地人民法院提起诉讼。  \n\n第八条 其他条款\n\n1. 本合同一式两份，甲乙双方各执一份，自签字后生效。'), -205.75024999599168)]
  self.vectorstore.similarity_search_with_relevance_scores(
No relevant docs were retrieved using the relevance score threshold 0.5


-----------------
您好! 如果您遇到了合同纠纷，通常可以采取以下几步来解决：

1. **核实合同条款**：首先，您需要检查合同的内容，看看是否有违约责任、赔偿金额等规定。
2. **联系对方**: 与对方沟通，了解他们的立场和需求。这可能会帮助您找到一个妥协的解决方案。
3. **寻求第三方仲裁**：如果双方无法达成协议，您可以考虑通过法律机构或专业仲裁机构进行调解。
4. **起诉**: 如果以上措施均不成功，您可以采取法律行动，向法院提出诉讼。

请注意，每种情况下的具体处理步骤可能会有所不同。在实际情况下，我建议您寻求专业的法律意见，以获得最准确和有效的指导。
